In [2]:
import requests

In [95]:
requests.get("http://127.0.0.1:2443")

<Response [200]>

In [126]:
requests.get("http://127.0.0.1:2443", verify=False)

<Response [200]>

In [122]:
requests.get("http://127.0.0.1:28080").text

'Hello world'

In [10]:
requests.get("http://127.0.0.1:18080").text

'Hello world'

In [1]:
import os
rfd, wfd = os.pipe()

In [3]:
os.close(0)

In [4]:
os.dup2(wfd, 1)

1

In [5]:
os.close(wfd)